In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model(f"./model_31-3-21.h5")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 240, 320, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 246, 326, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 120, 160, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 120, 160, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [3]:
import os
import cv2

In [4]:
crime = os.listdir("./1")
normal = os.listdir("./0")

In [8]:
crime_frames = []

for frame in crime:
    img = cv2.imread(f"./1/{frame}")
    crime_frames.append(img)
    
print("Done!")

Done!


In [9]:
normal_frames = []

for frame in normal:
    img = cv2.imread(f"./0/{frame}")
    normal_frames.append(img)
    
print("Done!")

Done!


In [11]:
crime_labels = [1]*len(crime)
normal_labels = [0]*len(normal)

In [24]:
def make_prediction(frame):
    out = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    out = cv2.resize(out, (320, 240)).astype("float32")
    pred = model.predict(np.expand_dims(out, axis=0))[0][0]
    
    return 1 if pred>0.5 else 0

In [25]:
make_prediction(crime_frames[1])

1

In [26]:
crime_predictions = []
normal_predictions = []

for frame in crime_frames:
    crime_predictions.append(make_prediction(frame))
    
for frame in normal_frames:
    normal_predictions.append(make_prediction(frame))

In [27]:
real_labels = crime_labels + normal_labels
pred_labels = crime_predictions + normal_predictions
videos = crime + normal

final = {
    "videos": videos,
    "true labels": real_labels,
    "predicted labels": pred_labels
}

In [36]:
import pandas as pd

pd.DataFrame(final).to_csv("testResults_x-4-21.csv", index=False)